In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import *
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle

In [2]:
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

In [3]:
def create_browser():
    """
    Function creating browser object.
    """
    # add options to disable pop-ups
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    # return browser object
    chrome = Chrome(options=chrome_options)
    return chrome

In [4]:
def get_user_page_first_time(chrome, user):
    """
    Function loading the basic page.
    """
    # load page
    chrome.get(f"https://www.filmweb.pl/user/{user}/films")
    # agree to terms of use
    chrome.find_element(By.ID, 'didomi-notice-agree-button').click()
    # wait until the advertisement ends
    time.sleep(35)
    chrome.maximize_window()
    time.sleep(10)

In [5]:
def get_user_page(chrome, user):
    """
    Function to get the user page.
    """
    # load page
    chrome.get(f"https://www.filmweb.pl/user/{user}/films")
    time.sleep(10)

In [6]:
def login_to_facebook(chrome, email, user_password):
    """
    Function logging to Facebook.
    """
    for i in range(20):
        chrome.execute_script(f"window.scrollBy(0,{i})","")
        
    chrome.find_element(By.CLASS_NAME, 'facebookLoginButton__button').click()
    
    # get window to scroll that showed up
    scrollable = chrome.find_element(By.CLASS_NAME, '_4t2a')
    chrome.execute_script("return arguments[0].scrollIntoView(true);", scrollable)
    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # click whatever
    button = chrome.find_element(By.CLASS_NAME, '_9xo5')
    button.click()
    
    # get all elements needed to log in
    login = chrome.find_element(By.ID, 'email')
    password = chrome.find_element(By.ID, 'pass')
    submit = chrome.find_element(By.ID, 'loginbutton')
    
    # enter credentials
    # login.click()
    login.send_keys(email)
    # password.click()
    password.send_keys(user_password)
    
    # click submit
    submit.click()
    
    time.sleep(10)

In [7]:
def get_genres_and_actors(chrome):
    div = chrome.find_elements(By.CLASS_NAME, 'preview__details')
    all_films = []
    for div_element in div:
        per_film = div_element.find_elements(By.CSS_SELECTOR, 'a')
        one_film_list = []
        for att in per_film:
            res = att.get_attribute('textContent')
            one_film_list.append(res)
        formatted = (tuple(one_film_list[:-2]), tuple(one_film_list[-2:]))
        all_films.append(formatted)
    return all_films

In [8]:
def get_ratings(chrome, user):
    """
    Function returning ratings from the first page.
    """
    
    # scroll down to load ratings
    for i in range(1000):
        chrome.execute_script(f"window.scrollBy(0,{i})","")
    
    # get ratings
    ratings = chrome.find_elements(By.CLASS_NAME, 'userRate__rate')
    ratings_list = [r.get_attribute('textContent') for r in ratings]

    # get film names
    film_names = chrome.find_elements(By.CLASS_NAME, 'preview__link')
    film_names_list = [f.get_attribute('textContent') for f in film_names]
    
    # get year of production
    preview_years = chrome.find_elements(By.CLASS_NAME, 'preview__year')
    preview_years_list = [p.get_attribute('textContent') for p in preview_years]
    
    # get genres and actors
    genres_actors_list = get_genres_and_actors(chrome)
    
    # get community ratings
    community_ratings = chrome.find_elements(By.CLASS_NAME, 'communityRatings__value')
    community_ratings_list = [r.get_attribute('textContent').strip() for r in community_ratings]
    community_ratings_list = community_ratings_list[::2]
    
    # number of ratings
    community_ratings_number = chrome.find_elements(By.CLASS_NAME, 'communityRatings__description')
    ratings_number_list = [r.get_attribute('textContent').strip() for r in community_ratings_number]
    ratings_number_list = [i[:i.find('ocen')].replace(' ', '') for i in ratings_number_list]
    ratings_number_list = list(filter(('chcezobaczy').__ne__, ratings_number_list))
    
    # add user name
    user_name_list = len(ratings) * [user]
    
    # list of tuples
    zipped_values = list(zip(user_name_list,
                             film_names_list, 
                             preview_years_list,
                             genres_actors_list,
                             community_ratings_list,
                             ratings_number_list,
                             ratings_list))
    
    
    
    return zipped_values
    

In [9]:
def get_all_user_ratings(chrome, user):
    """
    Function returning all ratings of given user.
    The results are written to pickle files.
    """
    # get ratings from the first page
    first = get_ratings(chrome, user)
    print(first)
    
    # prepare placeholders for ratings
    old = []
    new = first
    
    # starting page number
    page = 2
    
    # iterate as long as there are films ratings on the next pages
    while (len(new) > 0) and (old != new):
        print(page)
        chrome.get(f"https://www.filmweb.pl/user/{user}/films?page={page}")
        x = get_ratings(chrome, user)
        print(x)
        page += 1
        old, new = old + new, x
        
    if len(old) > 0:
        with open(f"{user}_full_data.pickle", 'wb') as handle:
            pickle.dump(old, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
def get_user_friends(chrome, user):
    """
    Function returning the list of user friends.
    """
    chrome.get(f"https://www.filmweb.pl/user/{user}/friends")
    friends = chrome.find_elements(By.CLASS_NAME, 'user__nick')
    friends_list = [f.get_attribute('textContent').strip() for f in friends]
    
    return friends_list

In [11]:
def get_ratings_for_all_friends(chrome, user):
    """
    Summary function: get all ratings for all friends.
    """
    friends_list = get_user_friends(chrome, user)
    for user in friends_list:
        print(f"Displaying results for user {user}")
        chrome.get(f"https://www.filmweb.pl/user/{user}/films")
        get_all_user_ratings(chrome, user)

# SCRAPPING YOURSELF AND YOUR FRIENDS

In [12]:
# EXAMPLE USE

# 1. create browser object.
chrome = create_browser()

# 2. load the first page, accept usage terms etc.
get_user_page_first_time(chrome, "<filmweb_user>")

# 3. login to Facebook. In case it fails, just do it by hand in this special browser window.
login_to_facebook(chrome, "<mail>", "<pass>")

# 4. congrats! you are ready to get the records
get_ratings_for_all_friends(chrome, "<filmweb_user>")

# 5. if something fails in between, you can get records just for one user
#get_all_user_ratings(chrome, user)

Displaying results for user sokoly35
[('sokoly35', 'Harry Potter i Zakon Feniksa', '2007', (('Familijny', 'Fantasy', 'Przygodowy'), ('Daniel Radcliffe', 'Rupert Grint')), '7,5', '322604', '6'), ('sokoly35', 'Gra tajemnic', '2014', (('Biograficzny', 'Dramat'), ('Benedict Cumberbatch', 'Keira Knightley')), '7,4', '185988', '10'), ('sokoly35', 'Gwiezdne wojny: Przebudzenie Mocy', '2015', (('Przygodowy', 'Sci-Fi'), ('Harrison Ford', 'Mark Hamill')), '7,3', '235657', '5'), ('sokoly35', 'Gwiezdne wojny: Część I - Mroczne widmo', '1999', (('Przygodowy', 'Sci-Fi'), ('Liam Neeson', 'Ewan McGregor')), '7,2', '277104', '8'), ('sokoly35', 'Bohemian Rhapsody', '2018', (('Biograficzny', 'Dramat', 'Muzyczny'), ('Rami Malek', 'Lucy Boynton')), '7,9', '214523', '8'), ('sokoly35', 'Jak rozpętałem drugą wojnę światową', '1969', (('Komedia', 'Przygodowy', 'Wojenny'), ('Marian Kociniak', 'Wirgiliusz Gryń')), '7,9', '275240', '9'), ('sokoly35', 'Hobbit: Bitwa Pięciu Armii', '2014', (('Fantasy', 'Przygodowy'

[('wgumiezny', 'After 4. Bez siebie nie przetrwamy', '2022', (('Melodramat',), ('Josephine Langford', 'Hero Fiennes-Tiffin')), '4,5', '2082', '10'), ('wgumiezny', 'After. Ocal mnie', '2021', (('Melodramat',), ('Josephine Langford', 'Hero Fiennes-Tiffin')), '4,4', '5174', '10'), ('wgumiezny', 'After 2', '2020', (('Melodramat',), ('Josephine Langford', 'Hero Fiennes-Tiffin')), '5,0', '10671', '10'), ('wgumiezny', 'After', '2019', (('Melodramat',), ('Josephine Langford', 'Hero Fiennes-Tiffin')), '5,2', '31355', '10'), ('wgumiezny', 'Na szczyt', '2020', (('Thriller',), ('Miguel Herrán', 'Luis Tosar')), '5,7', '3706', '10'), ('wgumiezny', 'Jak zostałem gangsterem. Historia prawdziwa', '2019', (('Sensacyjny',), ('Marcin Kowalczyk', 'Tomasz Włosok')), '7,4', '123801', '10')]
2
[('wgumiezny', 'After 4. Bez siebie nie przetrwamy', '2022', (('Melodramat',), ('Josephine Langford', 'Hero Fiennes-Tiffin')), '4,5', '2082', '10'), ('wgumiezny', 'After. Ocal mnie', '2021', (('Melodramat',), ('Josephin

[('Yoyuu', 'Glass Onion: Film z serii "Na noże"', '2022', (('Dramat', 'Komedia', 'Kryminał'), ('Daniel Craig', 'Edward Norton')), '6,7', '39711', '8'), ('Yoyuu', 'Na noże', '2019', (('Dramat', 'Komedia', 'Kryminał'), ('Daniel Craig', 'Chris Evans')), '7,8', '155198', '8'), ('Yoyuu', 'Tromeo i Julia', '1996', (('Czarna komedia',), ('Will Keenan', 'Jane Jensen')), '6,2', '1040', '6'), ('Yoyuu', 'Festen', '1998', (('Dramat',), ('Ulrich Thomsen', 'Henning Moritzen')), '7,7', '13869', '10'), ('Yoyuu', 'W trójkącie', '2022', (('Dramat', 'Komedia'), ('Harris Dickinson', 'Charlbi Dean')), '7,3', '6378', '9'), ('Yoyuu', 'Syrenkolandia', '2006', (('Animacja', 'Baśń'), ('Kelly Sheridan', 'Lee Tockar')), '6,3', '3089', '7'), ('Yoyuu', 'Barbie i podwodna tajemnica 2', '2012', (('Animacja', 'Dla dzieci'), ('Dominika Sell', 'Kelly Sheridan')), '6,4', '2531', '8'), ('Yoyuu', 'Barbie przedstawia Calineczkę', '2009', (('Animacja', 'Dla dzieci'), ('Anna Cummer', 'Ashleigh Ball')), '6,0', '2708', '6'), ('

[('Veshhen', 'Anioł śmierci', '2014', (('Dramat', 'Horror'), ('Phoebe Fox', 'Jeremy Irvine')), '5,2', '18739', '1'), ('Veshhen', 'Mechaniczna pomarańcza', '1971', (('Dramat', 'Sci-Fi'), ('Malcolm McDowell', 'Patrick Magee')), '7,9', '214205', '10'), ('Veshhen', 'Joker', '2019', (('Dramat', 'Kryminał', 'Psychologiczny'), ('Joaquin Phoenix', 'Robert De Niro')), '8,3', '299469', '8'), ('Veshhen', 'Forrest Gump', '1994', (('Dramat', 'Komedia'), ('Tom Hanks', 'Robin Wright')), '8,5', '981025', '7'), ('Veshhen', 'Flintstonowie', '1994', (('Familijny', 'Komedia'), ('John Goodman', 'Rick Moranis')), '5,7', '46147', '1')]
Displaying results for user julciak202
[('julciak202', 'Do ostatniej kości', '2022', (('Dramat', 'Horror', 'Romans'), ('Taylor Russell', 'Timothée Chalamet')), '6,6', '6915', '6'), ('julciak202', 'Glass Onion: Film z serii "Na noże"', '2022', (('Dramat', 'Komedia', 'Kryminał'), ('Daniel Craig', 'Edward Norton')), '6,7', '39711', '5'), ('julciak202', 'Spirited Away: W krainie B

[('bartosz_strzelecki', 'Ulepszenie', '2018', (('Akcja', 'Sci-Fi'), ('Logan Marshall-Green', 'Betty Gabriel')), '7,3', '45231', '8'), ('bartosz_strzelecki', 'Green Book', '2018', (('Dramat', 'Komedia'), ('Viggo Mortensen', 'Mahershala Ali')), '8,2', '214152', '8'), ('bartosz_strzelecki', 'Wściekłe psy', '1992', (('Thriller', 'Gangsterski'), ('Harvey Keitel', 'Michael Madsen')), '8,0', '216710', '8'), ('bartosz_strzelecki', 'Siła strachu', '2005', (('Thriller',), ('Robert De Niro', 'Dakota Fanning')), '7,2', '47152', '7'), ('bartosz_strzelecki', 'Klucz do koszmaru', '2005', (('Thriller',), ('Kate Hudson', 'Gena Rowlands')), '6,8', '44106', '7'), ('bartosz_strzelecki', 'Zapowiedź', '2009', (('Thriller', 'Sci-Fi'), ('Nicolas Cage', 'Chandler Canterbury')), '6,6', '139555', '7'), ('bartosz_strzelecki', '1984', '1984', (('Melodramat', 'Thriller', 'Sci-Fi'), ('John Hurt', 'Richard Burton')), '7,0', '12395', '8'), ('bartosz_strzelecki', 'Memento', '2000', (('Kryminał', 'Thriller'), ('Guy Pear

...

[('piotrr99', 'American Pie: Bractwo Beta', '2007', (('Komedia',), ('John White', 'Steve Talley')), '6,3', '64285', '5'), ('piotrr99', 'Wilk z Wall Street', '2013', (('Biograficzny', 'Komedia kryminalna'), ('Leonardo DiCaprio', 'Jonah Hill')), '7,7', '450391', '6'), ('piotrr99', 'Menu', '2022', (('Horror', 'Czarna komedia'), ('Ralph Fiennes', 'Anya Taylor-Joy')), '7,0', '11679', '8'), ('piotrr99', 'Troll', '2022', (('Fantasy', 'Przygodowy', 'Akcja'), ('Ine Marie Wilmann', 'Kim Falck')), '5,2', '11014', '4'), ('piotrr99', 'Ciało', '2003', (('Czarna komedia',), ('Rafał Królikowski', 'Emilia Krakowska')), '6,6', '55208', '6'), ('piotrr99', 'Truman Show', '1998', (('Dramat', 'Komedia', 'Sci-Fi'), ('Jim Carrey', 'Laura Linney')), '7,4', '504415', '9'), ('piotrr99', 'Straszny film', '2000', (('Komedia',), ('Anna Faris', 'Jon Abrahams')), '6,3', '271302', '4'), ('piotrr99', 'Gladiator', '1992', (('Dramat', 'Akcja', 'Sportowy'), ('James Marshall', 'Cuba Gooding Jr.')), '7,1', '6158', '8'), ('p

[('polko14', 'Shiny_Flakes: Nastoletni baron narkotykowy', '2021', (('Dokumentalny',), ('True crime', 'Maximilian Schmidt')), '7,0', '2898', '8'), ('polko14', 'The Kissing Booth 2', '2020', (('Komedia rom.', 'Dla młodzieży'), ('Joey King', 'Jacob Elordi')), '6,0', '19268', '7'), ('polko14', 'Do wszystkich chłopców: Zawsze i na zawsze', '2021', (('Romans', 'Dla młodzieży'), ('Lana Condor', 'Noah Centineo')), '5,8', '12941', '7'), ('polko14', 'Opiekunka: Demoniczna królowa', '2020', (('Horror', 'Komedia'), ('Samara Weaving', 'Judah Lewis')), '5,3', '11568', '7'), ('polko14', 'Lista priorytetów', '2020', (('Komedia',), ('Eli Brown', 'Madison Iseman')), '5,2', '2193', '5'), ('polko14', 'Do zaliczenia', '2013', (('Komedia',), ('Aubrey Plaza', 'Johnny Simmons')), '4,8', '21673', '4'), ('polko14', 'W lesie dziś nie zaśnie nikt', '2020', (('Horror',), ('Julia Wieniawa-Narkiewicz', 'Michał Lupa')), '4,4', '92529', '4'), ('polko14', 'Wszystkie jasne miejsca', '2020', (('Melodramat',), ('Elle Fan

[('kubawojcik01', 'Dom', '2022', (('Animacja', 'Czarna komedia'), ('Helena Bonham Carter', 'Matthew Goode')), '6,8', '7171', '10'), ('kubawojcik01', 'Shrek', '2001', (('Animacja', 'Familijny', 'Komedia'), ('Mike Myers', 'Eddie Murphy')), '7,8', '825390', '1'), ('kubawojcik01', 'Słoń', '2022', (('Melodramat',), ('Jan Hrynkiewicz', 'Paweł Tomaszewski')), '6,5', '1687', '5'), ('kubawojcik01', 'Hiacynt', '2021', (('Thriller',), ('Tomasz Ziętek', 'Hubert Miłkowski')), '6,4', '25153', '6'), ('kubawojcik01', 'Dobry opiekun', '2022', (('Dramat',), ('Jessica Chastain', 'Eddie Redmayne')), '6,4', '13999', '7'), ('kubawojcik01', 'Szarlatan', '2020', (('Dramat',), ('Ivan Trojan', 'Josef Trojan')), '6,7', '8863', '8'), ('kubawojcik01', 'Silent Twins', '2022', (('Biograficzny', 'Dramat'), ('Letitia Wright', 'Tamara Lawrance')), '7,0', '4885', '7'), ('kubawojcik01', 'Solaris', '1972', (('Dramat', 'Sci-Fi'), ('Tamara Ogorodnikova', 'Donatas Banionis')), '7,3', '10085', '8'), ('kubawojcik01', 'Zmierzch

[('szymanek_a', 'Rodzina od zaraz', '2018', (('Komedia obycz.',), ('Mark Wahlberg', 'Rose Byrne')), '7,3', '33041', ''), ('szymanek_a', 'Sekretne życie zwierzaków domowych 2', '2019', (('Animacja', 'Komedia', 'Przygodowy'), ('Patton Oswalt', 'Kevin Hart')), '6,7', '24088', '7'), ('szymanek_a', 'Parasite', '2019', (('Dramat',), ('Kang-ho Song', 'Seon-gyun Lee')), '8,1', '164632', '8'), ('szymanek_a', 'Coco', '2017', (('Animacja', 'Familijny', 'Przygodowy'), ('Anthony Gonzalez', 'Gael García Bernal')), '8,3', '157426', '9'), ('szymanek_a', 'Mamma Mia!', '2008', (('Musical', 'Komedia rom.'), ('Meryl Streep', 'Pierce Brosnan')), '6,6', '239954', '7'), ('szymanek_a', 'Projektantka', '2015', (('Dramat', 'Komedia'), ('Kate Winslet', 'Liam Hemsworth')), '7,5', '91130', '8'), ('szymanek_a', 'Kopciuszek', '1950', (('Animacja', 'Baśń'), ('Verna Felton', 'Luis Van Rooten')), '7,4', '53557', '8'), ('szymanek_a', 'Aladyn', '1992', (('Animacja', 'Baśń'), ('Robin Williams', 'Scott Weinger')), '7,4', '

[('paula2664', 'Pewnego razu... w Hollywood', '2019', (('Dramat', 'Kryminał'), ('Leonardo DiCaprio', 'Brad Pitt')), '7,3', '188075', '5'), ('paula2664', 'Nasze magiczne Encanto', '2021', (('Animacja', 'Familijny', 'Przygodowy'), ('Stephanie Beatriz', 'María Cecilia Botero')), '7,2', '32785', '7'), ('paula2664', 'Za wszelką cenę', '2004', (('Dramat',), ('Clint Eastwood', 'Hilary Swank')), '7,8', '273396', '7'), ('paula2664', 'Bohemian Rhapsody', '2018', (('Biograficzny', 'Dramat', 'Muzyczny'), ('Rami Malek', 'Lucy Boynton')), '7,9', '214523', '7'), ('paula2664', 'Adwokat diabła', '1997', (('Thriller',), ('Keanu Reeves', 'Al Pacino')), '7,9', '435180', '6'), ('paula2664', 'Sound of Metal', '2019', (('Dramat', 'Romans'), ('Riz Ahmed', 'Olivia Cooke')), '7,8', '31843', '7'), ('paula2664', 'Co w duszy gra', '2020', (('Animacja', 'Komedia', 'Przygodowy'), ('Jamie Foxx', 'Tina Fey')), '7,8', '32814', '7'), ('paula2664', 'Aida', '2020', (('Dramat',), ('Jasna Djuricic', 'Izudin Bajrović')), '8,

# SCRAPPING TOP 500 USERS

In [1]:
def get_most_active_users_one_page(chrome, page):
    if page == 1:
        chrome.get('https://www.filmweb.pl/users/ranking/usersMonthly')
    else:
        chrome.get(f"https://www.filmweb.pl/users/ranking/usersMonthly?page={page}")
    friends = chrome.find_elements(By.CLASS_NAME, 'rankingList__userName')
    friends_list = [f.get_attribute('textContent').strip() for f in friends]
    return friends_list

In [2]:
def get_500_most_active_users(chrome):
    users_list = []
    for page_num in range(10):
        users_per_page = get_most_active_users_one_page(chrome, page_num + 1)
        users_list = users_list + users_per_page
    return users_list

In [ ]:
active_500_users = get_500_most_active_users(chrome)

for user in active_500_users:
    get_user_page(chrome, user)
    print(f"Displaying results for user {user}")
    ratings = get_ratings(chrome, user)
    with open(f"{user}_full_data.pickle", 'wb') as handle:
        pickle.dump(ratings, handle, protocol=pickle.HIGHEST_PROTOCOL)